In [118]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

DF_LEN = 6

def get_table_with_links(driver):
    tiempo_inicial = time.time()
      # Abrir una nueva instancia del browser
    lista_proveedores_completos = pd.read_csv('contrataciones_por_proveedor.csv',delimiter=';', usecols = [0])
    proveedores = lista_proveedores_completos["Proveedor"].unique()
    print(proveedores)
    url = 'https://appscvsmovil.supercias.gob.ec/PortalInfor/consultaPrincipal.zul'
    driver.get(url)
    
    for proveedor in proveedores:
        
        print(url)
        
        try:
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//span[@class="z-radio"][3]/input'))).click()
            driver.find_element_by_xpath('//span[@class="z-radio"][3]/input').click() #click radio nombre
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="z-combobox-inp z-combobox-right-edge"]')))
            search_box = driver.find_element_by_xpath('//*[@class="z-combobox-inp z-combobox-right-edge"]')
            #search_box.click()
            #search_box.send_keys('tax')
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="z-combobox-inp z-combobox-right-edge"]'
        ))).send_keys(proveedor)
            sleep(1)
            WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//td[@class="z-button-cm"]'))).click()
            WebDriverWait(driver, 5).until(lambda driver: driver.current_url != url)

            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//a[7]'))).click()
        
            accionistas = []
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="z-treerow"]')))
            df = pd.DataFrame(columns=['identificacion', 'nombre', 'nacionalidad', 'tipo_inversion', 'capital', 'restriccion', 'nombre_proveedor'])
            for i in driver.find_elements_by_xpath('//*[contains(@class, "z-treerow")]'):
                accionista = []

                for index,j in enumerate(i.find_elements_by_xpath('td')):
                    if index>0: #para no obtener el numero de accionista.
                        try:
                            accionista.append(j.text)
                        except:
                            accionista.append("")

                if len(accionista) == DF_LEN:
                    accionista.append(proveedor)
                    accionistas.append(accionista)
                    df.loc[len(df)]=accionista
        
        except:
            print("Proveedor {} fallido".format(proveedor))
            df = pd.DataFrame({'identificacion': "", 'nombre': "", 'nacionalidad': "", 'tipo_inversion': "", 'capital': "", 'restriccion': "", 'nombre_proveedor': proveedor}, index=[0])
            df.to_csv("empresas.csv", index=False, header=False, mode = 'a', sep=";")
            driver.get(url)
            continue

        #submit_search = driver.find_element_by_xpath('//td[@class="z-button-cm"]')
        # //*[@class="z-treecell-cnt z-overflow-hidden"]
        print(accionistas)
        df.to_csv("empresas.csv", index=False, header=False, mode = 'a', sep=";")
        try:
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="z-window-modal-icon z-window-modal-close"]'))).click()
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="z-button-cm"]'))).click()
        except:
            driver.get(url)
            
            
    print("tiempo final:", time.time()-tiempo_inicial)
 
    driver.quit()
    #print(df)
    
 
options = Options()
#options.binary_location=r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe'
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#options.add_argument("--headless")
my_driver = webdriver.Chrome(options=options)
 
get_table_with_links(my_driver)

1     SUMINISTROS MEDICOS MANOSALVAS VILLAGOMEZ MV A...
2         MILENIO LIMPIEZA Y MANTENIMIENTO S.A. MILIMAN
3                      CHICAIZA ALVARADO PEDRO GIOVANNY
4                                   CONSORCIO CASUARINA
5                                          SALUMED S.A.
6                         CONSORCIO CEMENTERIO SUBURBIO
7       COMERCIALIZADORA NUTRI MED NUTRIMEDECUADOR S.A.
8                                        ARKITRUST S.A.
9                                    ROCHE ECUADOR S.A.
10            CAMPOSANTOS DEL ECUADOR S.A. CAMPOECUADOR
Name: Proveedor, dtype: object
https://appscvsmovil.supercias.gob.ec/PortalInfor/consultaPrincipal.zul
[['1713581740', 'ARIAS VINUEZA NORY TATIANA', 'ECUADOR', 'NACIONAL', '200.0000', 'N', 'SUMINISTROS MEDICOS MANOSALVAS VILLAGOMEZ MV ASOCIADOS CIA. LTDA.'], ['1703282234', 'MANOSALVAS ORTIZ POLIVIO ELICIO', 'ECUADOR', 'NACIONAL', '4,900.0000', 'N', 'SUMINISTROS MEDICOS MANOSALVAS VILLAGOMEZ MV ASOCIADOS CIA. LTDA.'], ['1704993532', 'VIL

In [98]:
contrataciones = pd.read_csv('contrataciones_por_proveedor.csv',sep=';', skiprows=1000, chunksize=1000)

In [123]:
lista_proveedores_completos = pd.read_csv('contrataciones_por_proveedor.csv',delimiter=';', usecols = [0])
len(lista_proveedores_completos["Proveedor"].unique())

3591

In [108]:
df = pd.read_csv('contrataciones_por_proveedor.csv',delimiter=';', usecols = [0])
df.head()

,Proveedor
0,MARTINEZ VELASQUEZ ISIDORO RICARDO
1,SUMINISTROS MEDICOS MANOSALVAS VILLAGOMEZ MV A...
2,MILENIO LIMPIEZA Y MANTENIMIENTO S.A. MILIMAN
3,CHICAIZA ALVARADO PEDRO GIOVANNY
4,CONSORCIO CASUARINA


In [119]:
df.describe()

,Proveedor
count,19456
unique,3591
top,ESCANDON CARRION JOSE AUGUSTO
freq,153
